In [8]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

import time
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any
from pathlib import Path
from IPython.display import clear_output
from collections import defaultdict

from qubx import lookup, logger, QubxLogConfig
from qubx.core.basics import TriggerEvent, Trade, MarketEvent, Instrument, SubscriptionType
from qubx.core.interfaces import IStrategyContext, IStrategy
from qubx.connectors.ccxt.ccxt_connector import CCXTExchangesConnector
from qubx.connectors.ccxt.ccxt_trading import CCXTTradingConnector
from qubx.utils.runner import get_account_config
from qubx.pandaz import scols
from qubx.backtester.simulator import SimulatedTrading
from qubx.utils.runner import run_ccxt_paper_trading
from qubx.utils.collections import TimeLimitedDeque

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.0 Live paper trading

In [ ]:
class TradeTestStrat(IStrategy):
    _data_counter: int = 0
    _data_to_buffer: dict[tuple[str, Instrument], TimeLimitedDeque]

    def on_init(self, ctx: IStrategyContext):
        # ctx.set_base_subscription(SubscriptionType.ORDERBOOK)
        # ctx.set_base_subscription(SubscriptionType.OHLC)
        # ctx.set_warmup(SubscriptionType.OHLC, "1h")
        ctx.set_base_subscription(SubscriptionType.TRADE)
        self._data_to_buffer = defaultdict(lambda: TimeLimitedDeque("1Min", lambda x: x.time, unit="ns"))

    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        self._data_counter += 1
        self._data_to_buffer[(data.type, data.instrument)].append(data.data)
        if self._data_counter % 1000 == 0:
            logger.debug(f"Processed {self._data_counter} data points")

    def on_universe_change(
        self, ctx: IStrategyContext, add_instruments: list[Instrument], rm_instruments: list[Instrument]
    ):
        if add_instruments:
            _sub_to_params = ctx.get_subscriptions(ctx.instruments[0])
            for sub, params in _sub_to_params.items():
                ctx.subscribe(add_instruments, sub, **params)

    def get_data(self, type: str, instrument: Instrument) -> list:
        return list(self._data_to_buffer[(type, instrument)])


ctx = run_ccxt_paper_trading(
    strategy=(stg := TradeTestStrat()),
    exchange="BINANCE.UM",
    symbols=["BTCUSDT", "ETHUSDT"],
    # symbols=["BTCUSDT", "ETHUSDT", "ADAUSDT", "XRPUSDT"],
    blocking=False,
)

In [ ]:
ctx.stop()

In [ ]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

In [ ]:
len(ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE))

In [ ]:
i1 = ctx.instruments[1]
obs = stg.get_data("ohlc", i1)
print(f"Instrument: {i1}")
for i in range(1, 5):
    print(obs[-i])

### Add trade subscription and remove it

In [ ]:
ctx.subscribe(ctx.instruments, SubscriptionType.TRADE)

In [ ]:
trades = stg.get_data("trade", ctx.instruments[0])
trades[-5:]

In [ ]:
stg.ctx.unsubscribe(ctx.instruments, SubscriptionType.TRADE)

### Add new instrument to the universe

In [ ]:
s1 = lookup.find_symbol("BINANCE.UM", "XRPUSDT"); assert s1 is not None
ctx.set_universe(list(set(ctx.instruments) | {s1}))

In [ ]:
new_instruments = ctx.instruments[-2:]
print(new_instruments)
ctx.set_universe(new_instruments)

In [ ]:
obs = stg.get_data("orderbook", s1)
obs[-1]

In [ ]:
s2 = lookup.find_symbol("BINANCE.UM", "ADAUSDT"); assert s2 is not None
stg.ctx.set_universe(list(set(stg.ctx.instruments) | {s2}))

In [ ]:
trades = stg.get_data("trade", s2)
trades[-5:]

In [ ]:
ctx.stop()

## 2.0 Live execution

In [ ]:
import os
import dotenv
from qubx import QubxLogConfig
from qubx.utils.runner import run_ccxt_trading


QubxLogConfig.set_log_level("DEBUG")


class TradeTestStrat(IStrategy):
    _data_counter: int = 0

    def on_init(self, ctx: IStrategyContext):
        ctx.set_base_subscription(SubscriptionType.OHLC, timeframe="1m")
        ctx.set_warmup(SubscriptionType.OHLC, "1h")
        ctx.set_event_schedule("@hourly")
    
    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        self._data_counter += 1
        if self._data_counter % 1000 == 0:
            logger.debug(f"Processed {self._data_counter} data points")

    def on_universe_change(
        self, ctx: IStrategyContext, add_instruments: list[Instrument], rm_instruments: list[Instrument]
    ):
        if add_instruments:
            _sub_to_params = ctx.get_subscriptions(ctx.instruments[0])
            for sub, params in _sub_to_params.items():
                ctx.subscribe(add_instruments, sub, **params)


dotenv.load_dotenv(
    "/mnt/HC_Volume_100695026/home/shared/devs/Qubx/debug/.env.binance.yuriy"
)

credentials = {
    "apiKey": os.getenv("BINANCE_KEY"),
    "secret": os.getenv("BINANCE_SECRET")
}

ctx = run_ccxt_trading(
    strategy=(stg := TradeTestStrat()),
    exchange="BINANCE",
    symbols=["BTCUSDT", "ETHUSDT"],
    credentials=credentials,
    blocking=False,
)

2024-11-13 18:27:32.007 [ ℹ️ ] BINANCE loading ...
2024-11-13 18:27:35.138 [ ℹ️ ] binance initialized - current time 2024-11-13T18:27:35.135090944
2024-11-13 18:27:35.138 [ ℹ️ ] Loading account data for Binance
2024-11-13 18:27:35.883 [ ⚠️ ] qubx.connectors.ccxt.ccxt_utils:ccxt_restore_position_from_deals:102 - Couldn't restore full deals history for BTCUSDT symbol. Qubx will use zero position !
2024-11-13 18:27:36.380 [ ⚠️ ] qubx.connectors.ccxt.ccxt_utils:ccxt_restore_position_from_deals:102 - Couldn't restore full deals history for ETHUSDT symbol. Qubx will use zero position !
2024-11-13 18:27:36.383 [ ℹ️ ] (StrategyContext) Start processing market data
2024-11-13 18:27:36.383 [ ℹ️ ] (StrategyContext) strategy is started in thread
2024-11-13 18:27:36.385 [ 🐞 ] Running OHLC warmup for {BINANCE:CRYPTO:BTCUSDT, BINANCE:CRYPTO:ETHUSDT} with period 1h
2024-11-13 18:27:36.385 [ 🐞 ] Listening to BTCUSDT executions
2024-11-13 18:27:36.386 [ 🐞 ] Listening to ETHUSDT executions


2024-11-13 18:27:39.129 [ ℹ️ ] BINANCE:CRYPTO:BTCUSDT: loaded 60 1m bars
2024-11-13 18:27:39.130 [ 🐞 ] Invoking TradeTestStrat on_fit
2024-11-13 18:27:39.131 [ 🐞 ] TradeTestStrat is fitted
2024-11-13 18:27:39.159 [ ℹ️ ] BINANCE:CRYPTO:ETHUSDT: loaded 60 1m bars
2024-11-13 18:27:39.160 [ 🐞 ] Listening to BTCUSDT,ETHUSDT ohlc (warmup_period=1h)
2024-11-13 18:29:01.210 [ 🐞 ] (StrategyContext) sending limit buy for 0.00019 of BTCUSDT @ 88000.0 ...
2024-11-13 18:29:01.451 [ ℹ️ ] (CCXTSyncTradingConnector) New order [32397996623] LIMIT BUY 0.00019 of BTCUSDT @ 88000.0 (GTC) [NEW]
2024-11-13 18:29:01.467 [ 🐞 ] Order 32397996623 LIMIT BUY 0.00019 of BINANCE:CRYPTO:BTCUSDT -> OPEN
2024-11-13 18:29:01.468 [ 🐞 ] [32397996623 / TradeTestStrat_BTCUSDT_17315225413] : LIMIT BUY 0.00019 of BTCUSDT  @ 88000.0 -> [OPEN]
2024-11-13 18:29:27.774 [ ℹ️ ] Canceling order 32397996623 ...
2024-11-13 18:29:28.030 [ 🐞 ] Order 32397996623 LIMIT BUY 0.00019 of BINANCE:CRYPTO:BTCUSDT -> CANCELED
2024-11-13 18:29:28

In [10]:
ctx.get_total_capital()

17.05809094

In [14]:
ctx.ohlc(ctx.instruments[0]).pd().tail()

,open,high,low,close,volume,bought_volume
timestamp,,,,,,
2024-11-13 18:24:00,92810.29,92810.30,92672.00,92693.99,3.667641e+06,1.209412e+06
2024-11-13 18:25:00,92693.99,92751.29,92540.01,92599.99,7.754366e+06,5.357839e+06
2024-11-13 18:26:00,92599.99,92618.32,92504.86,92572.22,3.638763e+06,1.333537e+06
2024-11-13 18:27:00,92572.23,92661.25,92564.95,92659.10,1.885044e+06,5.906647e+05
2024-11-13 18:28:00,92659.09,92712.00,92618.01,92626.88,1.449182e+06,4.500472e+05


In [15]:
o = ctx.trade(ctx.instruments[0], amount=0.00019, price=88000)
o

Order(id='32397996623', type='LIMIT', instrument=BINANCE:CRYPTO:BTCUSDT, time=Timestamp('2024-11-13 18:29:01.332000'), quantity=0.00019, price=88000.0, side='BUY', status='NEW', time_in_force='GTC', client_id='TradeTestStrat_BTCUSDT_17315225413', cost=0.0, options={})

In [16]:
ctx.cancel_order(o.id)

In [17]:
ctx.subscribe(ctx.instruments, SubscriptionType.TRADE)

In [ ]:
ctx.unsubscribe(ctx.instruments, SubscriptionType.TRADE)

In [ ]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

In [ ]:
ctx.set_universe([ctx.instruments[0]])

In [18]:
ctx.subscribe(ctx.instruments, SubscriptionType.ORDERBOOK)

In [ ]:
ctx.unsubscribe(ctx.instruments, SubscriptionType.ORDERBOOK)

In [22]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

[[2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180607,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180608,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180609,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180610,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180611]

In [23]:
ctx.get_data(ctx.instruments[0], SubscriptionType.ORDERBOOK)[-5:]

[[2024-11-13T18:30:43.213000000] 92687.4 (5.00577) | 92687.41 (2.88649),
 [2024-11-13T18:30:43.313000000] 92687.4 (5.36551) | 92687.41 (0.74832),
 [2024-11-13T18:30:43.413000000] 92687.4 (4.57564) | 92687.41 (0.41051),
 [2024-11-13T18:30:43.513000000] 92687.4 (4.58619) | 92687.41 (0.55437),
 [2024-11-13T18:30:43.613000000] 92687.4 (4.58658) | 92687.41 (0.55437)]

In [25]:
i1 = ctx.instruments[0]
q = ctx.quote(i1)
q

[2024-11-13T18:31:11.513000000]	92536.88000 (1.6) | 92536.89000 (2.6)

In [26]:
ctx.stop()

In [ ]:
pd.Timedelta("1h").seconds

### Tmp stuff

In [ ]:
from qubx.connectors.ccxt.ccxt_customizations import BinanceQV


binance = BinanceQV(credentials)

In [ ]:
import asyncio

ohlcv = None

async def example():
    global ohlcv
    subscriptions = [["BTCUSDT", "1m"]]
    try:
        while True:
            ohlcv = await binance.watch_ohlcv_for_symbols(subscriptions)
            print(pd.Timestamp(list(list(ohlcv.values())[0].values())[0][0][0], unit='ms'))
            print(ohlcv)
    except asyncio.CancelledError:
        print("Keyboard interrupt received, exiting...")

await example()